# Men from Mars, Women from Venus: both can code .NET in Jupyter

@rondagdag

### .NET Interactive https://github.com/dotnet/interactive

### To install in raspberry pi
```
curl -L https://raw.githubusercontent.com/colombod/pi-top/master/setup-device.sh | bash -e
source .jupyter_venv/bin/activate
jupyter lab --no-browser

```

Rasperry Pi Sense Hat installation instructions
https://www.raspberrypi.org/documentation/hardware/sense-hat/

In [1]:
#!lsmagic

The below script needs to be able to find the current output cell; this is an easy method to get it.

.NET kernel #!csharp , #!c# , #!C# Run the code that follows using the csharp kernel. #!fsharp , #!f# , #!F# Run the code that follows using the fsharp kernel. #!pwsh , #!powershell Run the code that follows using the pwsh kernel. #!javascript , #!js Run the code that follows using the javascript kernel. #!html Run the code that follows using the html kernel. #!value Stores a value #!lsmagic List the available magic commands / directives #!markdown Convert the code that follows from Markdown into HTML #!time Time the execution of the following code in the submission. #!log Enables session logging. #!about Show version and build information #!connect Connects additional subkernels named-pipe Connects to a kernel using named pipes signalr Connects to a kernel using SignalR

csharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

fsharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

pwsh kernel #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

javascript kernel #!lsmagic List the available magic commands / directives

html kernel #!lsmagic List the available magic commands / directives

value kernel #!lsmagic List the available magic commands / directives

In [2]:
#!fsharp

"Hello from F#" |> Console.WriteLine
let x = 10
let y = 15

Hello from F#


In [3]:
#!share --from fsharp x
x

10

In [4]:
#r "nuget:System.Device.Gpio,1.1.0-prerelease.20276.1"
#r "nuget:IoT.Device.Bindings,1.1.0-prerelease.20276.1"
#r "nuget:UnitsNet,4.68.0"

Installed package System.Device.Gpio version 1.1.0-prerelease.20276.1

Installed package UnitsNet version 4.68.0

Installed package IoT.Device.Bindings version 1.1.0-prerelease.20276.1

### Display the html

In [5]:
var pocketView = span(a[href:@"https://github.com/rondagdag/dotnetcode-jupyter-talk", target: "blank"](b("Presentation")));
display(pocketView)

Presentation

In [6]:
var someHtml = HTML("<b style=\"color:blue\">Sensor data!</b>");
var tempOutput = display(someHtml);

98.72749633789061

In [7]:
using Iot.Device.SenseHat;
using System;
var temperatures = new List<(int idx, double val)>();

async Task readTemperature(int count = 10){   
    using (var sh = new SenseHat())
    {
        var initcount = temperatures.Count;
        for (var i = initcount; i < initcount + count; i++){        
            temperatures.Add((i, sh.Temperature2.DegreesFahrenheit));
            tempOutput.Update(HTML($"<b style=\"color:green\">{sh.Temperature2.DegreesFahrenheit}</b>"));
            await Task.Delay(500);
        }
    }
}

### Read Temperatures

In [8]:
await readTemperature(15);

In [9]:
using XPlot.Plotly;

In [10]:
var values = temperatures.Select(i => i.val);
values

index,value
0,98.69374771118163
1,98.71249999999999
2,98.70500183105467
3,98.70500183105467
4,98.70125274658201
5,98.70875091552733
6,98.68999862670897
7,98.68624954223631
8,98.70125274658201
9,98.71624908447265


In [11]:
var openSeries = new Graph.Scatter
{
    name = "Open",
    x = temperatures.Select(i => i.idx),
    y = values
};


var chart = Chart.Plot(new[] {openSeries});
chart.WithTitle("Open");

### Display the chart

In [12]:
display(chart)

### Instead of adding to list, let's setup Asycnronous Streams in C#

In [13]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            await Task.Delay(1000);
            yield return (sh.Temperature2.DegreesFahrenheit);
        }
    };
}

In [14]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

Installed package Microsoft.Data.Analysis version 0.4.0

In [15]:
var output = display("Counting...");
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<double> fahrenheit = new PrimitiveDataFrameColumn<double>("Fahrenheit"); // Makes a column of length 3. Filled with nulls initially

await foreach (var result in ReadTemperatureAsync(15))
{
   output.Update(result);
   dateTimes.Append(DateTime.Now);
   fahrenheit.Append(result);
}

98.93375091552733

### create a new Dataframe with 2 columns

In [16]:
DataFrame df =  new DataFrame(dateTimes, fahrenheit);

In [17]:
df

Columns,Rows
"[ [ 2020-08-15 14:05:30Z, 2020-08-15 14:05:32Z, 2020-08-15 14:05:33Z, 2020-08-15 14:05:34Z, 2020-08-15 14:05:35Z, 2020-08-15 14:05:36Z, 2020-08-15 14:05:37Z, 2020-08-15 14:05:38Z, 2020-08-15 14:05:39Z, 2020-08-15 14:05:40Z, 2020-08-15 14:05:41Z, 2020-08-15 14:05:42Z, 2020-08-15 14:05:43Z, 2020-08-15 14:05:44Z, 2020-08-15 14:05:45Z ], [ 98.89624633789062, 98.90000228881834, 98.88499908447264, 98.903751373291, 98.92625274658202, 98.93375091552733, 98.92250366210936, 98.94874725341796, 98.94874725341796, 98.90750045776366, 98.93375091552733, 98.960001373291, 98.97875366210937, 98.95249633789061, 98.93375091552733 ] ]","[ [ 2020-08-15 14:05:30Z, 98.89624633789062 ], [ 2020-08-15 14:05:32Z, 98.90000228881834 ], [ 2020-08-15 14:05:33Z, 98.88499908447264 ], [ 2020-08-15 14:05:34Z, 98.903751373291 ], [ 2020-08-15 14:05:35Z, 98.92625274658202 ], [ 2020-08-15 14:05:36Z, 98.93375091552733 ], [ 2020-08-15 14:05:37Z, 98.92250366210936 ], [ 2020-08-15 14:05:38Z, 98.94874725341796 ], [ 2020-08-15 14:05:39Z, 98.94874725341796 ], [ 2020-08-15 14:05:40Z, 98.90750045776366 ], [ 2020-08-15 14:05:41Z, 98.93375091552733 ], [ 2020-08-15 14:05:42Z, 98.960001373291 ], [ 2020-08-15 14:05:43Z, 98.97875366210937 ], [ 2020-08-15 14:05:44Z, 98.95249633789061 ], [ 2020-08-15 14:05:45Z, 98.93375091552733 ] ]"


In [18]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 5;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [19]:
df

0,2020-08-15 14:05:30Z,98.89624633789062
1,2020-08-15 14:05:32Z,98.90000228881834
2,2020-08-15 14:05:33Z,98.88499908447264
3,2020-08-15 14:05:34Z,98.903751373291
4,2020-08-15 14:05:35Z,98.92625274658202


In [20]:
var a = df.Rows.Take(2);
a

index,value
0,"[ 2020-08-15 14:05:30Z, 98.89624633789062 ]"
1,"[ 2020-08-15 14:05:32Z, 98.90000228881834 ]"


### Count the number of rows

In [21]:
var total = df.Rows.Count();
total

15

In [22]:
PrimitiveDataFrameColumn<double> celcius = new PrimitiveDataFrameColumn<double>("Celcius", total);
var newDf = df.Clone();
newDf.Columns.Insert(2, celcius);
newDf

0,2020-08-15 14:05:30Z,98.89624633789062,<null>
1,2020-08-15 14:05:32Z,98.90000228881834,<null>
2,2020-08-15 14:05:33Z,98.88499908447264,<null>
3,2020-08-15 14:05:34Z,98.903751373291,<null>
4,2020-08-15 14:05:35Z,98.92625274658202,<null>


In [23]:
var info = newDf.Info();
info

0,DataType,System.DateTime,System.Double,System.Double
1,Length (excluding null values),15,15,0


In [24]:
celcius.FillNulls(0, inPlace: true);
newDf

0,2020-08-15 14:05:30Z,98.89624633789062,0
1,2020-08-15 14:05:32Z,98.90000228881834,0
2,2020-08-15 14:05:33Z,98.88499908447264,0
3,2020-08-15 14:05:34Z,98.903751373291,0
4,2020-08-15 14:05:35Z,98.92625274658202,0


In [27]:
DataFrameRow row0 = newDf.Rows[0];
row0

index,type,value
0,System.DateTime,2020-08-15 14:05:30Z
1,System.Double,98.89624633789062
2,System.Double,0


### Convert Fahrenheit and fill the Celcius

In [28]:
newDf.Columns["Celcius"] = (fahrenheit - 32) * 5/9;
newDf

0,2020-08-15 14:05:30Z,98.89624633789062,37.16458129882812
1,2020-08-15 14:05:32Z,98.90000228881834,37.16666793823241
2,2020-08-15 14:05:33Z,98.88499908447264,37.15833282470702
3,2020-08-15 14:05:34Z,98.903751373291,37.168750762939446
4,2020-08-15 14:05:35Z,98.92625274658202,37.1812515258789


In [29]:
var celciusMean = newDf.Columns["Celcius"].Mean();
celciusMean

37.18263905843098

In [30]:
Chart.Plot(
    new Graph.Scatter()
    {
        x = newDf.Columns["DateTimes"],
        y = newDf.Columns["Celcius"]
    }
)

In [32]:
var tempData = new List<double>();

async Task readTempData(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){        
            if(tempData.Count > 10){
                tempData.RemoveAt(0);
            }
            tempData.Add(sh.Temperature2.DegreesFahrenheit);
            Console.WriteLine(sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(500);
        }
    }
}

In [37]:
#!js 

if (typeof (notebookScope.interval) !== 'undefined') {
    clearInterval(notebookScope.interval);
}

notebookScope.plot = (sgvSelector, variableName) => {   
    let dtree_require = require.config({
        paths: {
            d3: "https://d3js.org/d3.v5.min"
        }
    });
    dtree_require(["d3"], function (d3) {       
        let svg = d3.select(sgvSelector);
        svg.selectAll("defs").remove();
        svg.selectAll("g").remove();
        
        var margin = {top: 0, right: 30, bottom: 30, left: 40},
            width = 260 - margin.left - margin.right,
            height = 200 - margin.top - margin.bottom;
        
        let container = d3
            .select(sgvSelector);
        
         var width = 260 
            scaleFactor = 20, 
            barHeight = 30;
         
         var graph = d3.select("body")
            .append("svg")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
            .append("g")
                .attr("transform",
                      "translate(" + margin.left + "," + margin.top + ")");
        
        // Add X axis
          var x = d3.scaleLinear()
            .domain([0, 10])
            .range([ width, 0 ]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + "," + height + ")")
            .call(d3.axisBottom(x));

      // Add Y axis
          var y = d3.scaleLinear()
            .domain([99, 101])
            .range([ height, 0]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + ", 0)")
            .call(d3.axisLeft(y));

        updateD3Rendering = () => interactive.csharp.getVariable(variableName)
            .then(data => {
                console.log(data);
                container
                    .selectAll(".points")
                    .data(data, (d, i) => i)
                    .join(
                        enter => enter.append("circle")
                            .attr("cx", (d, i) => { console.log("i:" + i); return margin.left + x(i); } )
                            .attr("cy", function (d) { console.log("d:" + d); return y(d); } )
                            .attr("r", 2.5)
                            .style("fill", "#69b3a2")
                            .transition()
                            .duration(2000)                            
                            .attr("r", 0 ).remove(),
                        update => update
                            .transition()
                             .attr("class", "line")
                             .attr("d", valueline)
                            );
            });
        notebookScope.interval = setInterval(() => updateD3Rendering(), 1000);
    });
}

### Plot and call javascript 

In [38]:
#!html
<svg id="dataPlot1" width=500 height=200>
</svg>

#!js
notebookScope.plot("svg#dataPlot1", "tempData")

In [39]:
await readTempData(50);

99.24500045776367
99.25624771118163
99.25624771118163
99.26375274658201
99.26375274658201
99.27874908447265
99.28999633789061
99.31249771118163
99.31249771118163
99.297501373291
99.27874908447265
99.27499999999999
99.27874908447265
99.28624725341795
99.27499999999999
99.26000366210937
99.241251373291
99.26750183105467
99.26000366210937
99.26375274658201
99.27499999999999
99.28624725341795
99.29375228881835
99.30499954223632
99.30874862670898
99.297501373291
99.297501373291
99.30874862670898
99.31625366210936
99.33124999999998
99.32000274658202
99.32000274658202
99.32750091552734
99.34249725341796
99.34624633789062
99.3387481689453
99.33499908447264
99.3387481689453
99.353751373291
99.33499908447264
99.33124999999998
99.32375183105468
99.32375183105468
99.32750091552734
99.34624633789062
99.34249725341796
99.34249725341796
99.3387481689453
99.35000228881835
99.38000183105467


In [40]:
tempData.Count

11

In [41]:
using System;
using System.Drawing;
async Task readTempCondition(int count = 10, double maxTemp = 100){   
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    using (var sh = new SenseHat())
    {
        ledMatrix.Fill(Color.Black);
        for (var i = 0; i < count; i++){
            var temp = sh.Temperature2.DegreesFahrenheit;
            Console.WriteLine(temp);
            if (temp > maxTemp) {
                ledMatrix.Fill(Color.Red);
            } 
            await Task.Delay(500);
        }
    }
}

### read temperature 50 times and add max temp condition

In [45]:
await readTempCondition(15, 99.75);

99.75874862670898
99.75499954223632
99.75499954223632
99.747501373291
99.71750183105468
99.70624771118163
99.71000366210936
99.71375274658202
99.72125091552734
99.725
99.72874908447264
99.73624725341796
99.73624725341796
99.73999633789062
99.74375228881834


### clear the ledMatrix to "black"

In [46]:
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}


## @rondagdag

## https://www.linkedin.com/in/rondagdag